In [2]:
import numpy as np
import matplotlib.pyplot as plt
import segyio
import matplotlib.patches as patches
from shapely.geometry import Polygon
import pandas as pd
import re

In [3]:
def read_seismic_cube(file_path):
    """
    :param file_path: Путь к файлу
    :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
    а также словарю сейсмотрасс, где ключ это координата пересечения Inlines и Xlines, а значение это список значений сейсмотрасс
    """
    with segyio.open(file_path, 'r') as segyfile:
        x = list(segyfile.ilines)
        y = list(segyfile.xlines)
        z = list(segyfile.samples)
        cube = segyio.tools.cube(segyfile)
        traces = {f"{x[i]}_{y[j]}": cube[i][j][:] for i in range(len(x)) for j in range(len(y))}
    return x, y, z, traces

In [4]:
file_path = 'C:/HV/Seismic/Cube_TWT.segy'
ilines, xlines, samples_0, traces = read_seismic_cube(file_path)
samples = [int(i) for i in samples_0]